## Tasks

1 - Generar script para levantar el JSON

In [28]:
import json
import pandas as pd

In [49]:
def up_json(file):
    with open(file, 'r') as file2:
    
        data = json.load(file2)
        df = pd.DataFrame([data])
        df.to_csv("df.csv", index=False)
    return df


In [50]:
a = up_json("example.json")
a

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.75,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import datetime as dt
import seaborn as sns
from scipy import stats

In [99]:
def up_preprocesing(data_train, data_test):
    
    # Identificando la data de train y de test, para posteriormente unión y separación
    data_train['Set'] = 'train'
    data_test['Set'] = 'test'
    data = pd.concat([data_train, data_test], ignore_index=True, sort=False)
    
    #Feature engineering años del establecimiento
    
    data['Outlet_Establishment_Year'] = 2020 - data['Outlet_Establishment_Year']
    
    #LIMPIEZA: Unificando etiquetas para 'Item_Fat_Content'
    
    data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'low fat':  'Low Fat', 'LF': 'Low Fat', 'reg': 'Regular'})
    
    #LIMPIEZA: de faltantes en el peso de los productos
    
    productos = list(data[data['Item_Weight'].isnull()]['Item_Identifier'].unique())
    for producto in productos:
        moda = (data[data['Item_Identifier'] == producto][['Item_Weight']]).mode().iloc[0,0]
        data.loc[data['Item_Identifier'] == producto, 'Item_Weight'] = moda
    
    #LIMPIEZA: de faltantes en el tamaño de las tiendas
    outlets = list(data[data['Outlet_Size'].isnull()]['Outlet_Identifier'].unique())
    for outlet in outlets:
        data.loc[data['Outlet_Identifier'] == outlet, 'Outlet_Size'] =  'Small'
    
    # FEATURES ENGINEERING: creando categorías para 'Item_Type'
    data['Item_Type'] = data['Item_Type'].replace({'Others': 'Non perishable', 'Health and Hygiene': 'Non perishable', 'Household': 'Non perishable',
     'Seafood': 'Meats', 'Meat': 'Meats',
     'Baking Goods': 'Processed Foods', 'Frozen Foods': 'Processed Foods', 'Canned': 'Processed Foods', 'Snack Foods': 'Processed Foods',
     'Breads': 'Starchy Foods', 'Breakfast': 'Starchy Foods',
     'Soft Drinks': 'Drinks', 'Hard Drinks': 'Drinks', 'Dairy': 'Drinks'})

    # FEATURES ENGINEERING: asignación de nueva categorías para 'Item_Fat_Content'
    data.loc[data['Item_Type'] == 'Non perishable', 'Item_Fat_Content'] = 'NA'
    
    #FEATURES ENGINEERING: Codificando los niveles de precios de los productos
    pd.qcut(data['Item_MRP'], 4,).unique()
    data['Item_MRP'] = pd.qcut(data['Item_MRP'], 4, labels = [1, 2, 3, 4])
    
    # Se utiliza una copia de data para separar los valores codificados en un dataframe distinto.
    
    dataframe = data.drop(columns=['Item_Type', 'Item_Fat_Content']).copy()
    
    # Codificación de variables ordinales
    dataframe['Outlet_Size'] = dataframe['Outlet_Size'].replace({'High': 2, 'Medium': 1, 'Small': 0})
    dataframe['Outlet_Location_Type'] = dataframe['Outlet_Location_Type'].replace({'Tier 1': 2, 'Tier 2': 1, 'Tier 3': 0}) # Estas categorias se ordenaron asumiendo la categoria 2 como más lejos
    
    #FEATURES ENGINEERING: Codificación de variables nominales
    dataframe = pd.get_dummies(dataframe, columns=['Outlet_Type'])
    
    # Eliminación de variables que no contribuyen a la predicción por ser muy específicas
    dataset = dataframe.drop(columns=['Item_Identifier', 'Outlet_Identifier'])

    # División del dataset de train y test
    df_train = dataset.loc[data['Set'] == 'train']
    df_test = dataset.loc[data['Set'] == 'test']

    # Eliminando columnas sin datos
    df_train.drop(['Set'], axis=1, inplace=True)
    df_test.drop(['Item_Outlet_Sales','Set'], axis=1, inplace=True)

    # Guardando los datasets
    df_train.to_csv("train_final.csv", index=False)
    df_test.to_csv("test_final.csv", index=False)
    

In [101]:
data_train = pd.read_csv('../data/Train_BigMart.csv') 
data_test = pd.read_csv('../data/Test_BigMart.csv')

In [102]:
x = up_preprocesing(data_train, data_test)
x

C:\Users\AF01975\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [88]:
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
from sklearn import metrics
from sklearn.linear_model import LinearRegression
import pickle


In [127]:
def training_process(train):
    df_train = pd.read_csv(train)
    seed = 28
    model = LinearRegression()

    # División de dataset de entrenaimento y validación
    X = df_train.drop(columns='Item_Outlet_Sales')#[['Item_Weight', 'Item_MRP', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type']] # .drop(columns='Item_Outlet_Sales')
    x_train, x_val, y_train, y_val = train_test_split(X, df_train['Item_Outlet_Sales'], test_size = 0.3, random_state=seed)

    # Entrenamiento del modelo
    model.fit(x_train,y_train)

    # Predicción del modelo ajustado para el conjunto de validación
    pred = model.predict(x_val)

    # Cálculo de los errores cuadráticos medios y Coeficiente de Determinación (R^2)
    mse_train = metrics.mean_squared_error(y_train, model.predict(x_train))
    R2_train = model.score(x_train, y_train)
    print('Métricas del Modelo:')
    print('ENTRENAMIENTO: RMSE: {:.2f} - R2: {:.4f}'.format(mse_train**0.5, R2_train))

    mse_val = metrics.mean_squared_error(y_val, pred)
    R2_val = model.score(x_val, y_val)
    print('VALIDACIÓN: RMSE: {:.2f} - R2: {:.4f}'.format(mse_val**0.5, R2_val))

    print('\nCoeficientes del Modelo:')
    # Constante del modelo
    print('Intersección: {:.2f}'.format(model.intercept_))

    # Coeficientes del modelo
    coef = pd.DataFrame(x_train.columns, columns=['features'])
    coef['Coeficiente Estimados'] = model.coef_
    print(coef, '\n')
    
    with open('model.pkl', 'wb') as archivo_pkl:
        pickle.dump(model, archivo_pkl)

In [128]:
t = training_process("train_final.csv")
t

Métricas del Modelo:
ENTRENAMIENTO: RMSE: 1169.35 - R2: 0.5284
VALIDACIÓN: RMSE: 1146.66 - R2: 0.5526

Coeficientes del Modelo:
Intersección: 253.70
                        features  Coeficiente Estimados
0                    Item_Weight              -2.332949
1                Item_Visibility            -311.774516
2                       Item_MRP             825.276595
3      Outlet_Establishment_Year             -10.632046
4                    Outlet_Size             102.518103
5           Outlet_Location_Type              27.760861
6      Outlet_Type_Grocery Store           -1664.691331
7  Outlet_Type_Supermarket Type1             191.570173
8  Outlet_Type_Supermarket Type2            -242.596116
9  Outlet_Type_Supermarket Type3            1715.717274 



In [129]:
def prediction(data, modelo):
    #Feature engineering años del establecimiento
    data = pd.read_csv(data)
    data['Outlet_Establishment_Year'] = 2020 - data['Outlet_Establishment_Year']
    
    #LIMPIEZA: Unificando etiquetas para 'Item_Fat_Content'
    
    data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'low fat':  'Low Fat', 'LF': 'Low Fat', 'reg': 'Regular'})
    
    #LIMPIEZA: de faltantes en el peso de los productos
    
    productos = list(data[data['Item_Weight'].isnull()]['Item_Identifier'].unique())
    for producto in productos:
        moda = (data[data['Item_Identifier'] == producto][['Item_Weight']]).mode().iloc[0,0]
        data.loc[data['Item_Identifier'] == producto, 'Item_Weight'] = moda
    
    #LIMPIEZA: de faltantes en el tamaño de las tiendas
    outlets = list(data[data['Outlet_Size'].isnull()]['Outlet_Identifier'].unique())
    for outlet in outlets:
        data.loc[data['Outlet_Identifier'] == outlet, 'Outlet_Size'] =  'Small'
    
    # FEATURES ENGINEERING: creando categorías para 'Item_Type'
    data['Item_Type'] = data['Item_Type'].replace({'Others': 'Non perishable', 'Health and Hygiene': 'Non perishable', 'Household': 'Non perishable',
     'Seafood': 'Meats', 'Meat': 'Meats',
     'Baking Goods': 'Processed Foods', 'Frozen Foods': 'Processed Foods', 'Canned': 'Processed Foods', 'Snack Foods': 'Processed Foods',
     'Breads': 'Starchy Foods', 'Breakfast': 'Starchy Foods',
     'Soft Drinks': 'Drinks', 'Hard Drinks': 'Drinks', 'Dairy': 'Drinks'})

    # FEATURES ENGINEERING: asignación de nueva categorías para 'Item_Fat_Content'
    data.loc[data['Item_Type'] == 'Non perishable', 'Item_Fat_Content'] = 'NA'
    
    #FEATURES ENGINEERING: Codificando los niveles de precios de los productos
    intervalos = [(31.289, 94.012), (94.012, 142.247), (142.247, 185.856),(185.856, 266.888)]
    valores_asignados = [1, 2, 3, 4]

    # Crear una nueva columna que contenga los valores asignados según los intervalos
    data['Item_MRP'] = pd.cut(data['Item_MRP'], bins=[intervalo[0] for intervalo in intervalos] + [float('inf')], labels=valores_asignados, right=False)
    
    # Se utiliza una copia de data para separar los valores codificados en un dataframe distinto.
    
    dataframe = data.drop(columns=['Item_Type', 'Item_Fat_Content']).copy()
    
    # Codificación de variables ordinales
    dataframe['Outlet_Size'] = dataframe['Outlet_Size'].replace({'High': 2, 'Medium': 1, 'Small': 0})
    dataframe['Outlet_Location_Type'] = dataframe['Outlet_Location_Type'].replace({'Tier 1': 2, 'Tier 2': 1, 'Tier 3': 0}) # Estas categorias se ordenaron asumiendo la categoria 2 como más lejos
    
    #FEATURES ENGINEERING: Codificación de variables nominales
    todos_valores = ['Grocery Store', 'Supermarket Type1', 'Supermarket Type2', 'Supermarket Type3']

    # Agregar columnas dummy faltantes con valores inicializados en 0
    for valor in todos_valores:
        dataframe[f'Outlet_Type_{valor}'] = 0

    # Marcar con 1 la columna correspondiente al valor presente en el registro de ejemplo
    dataframe.loc[0, f'Outlet_Type_{dataframe["Outlet_Type"].iloc[0]}'] = 1

    
    # Eliminación de variables que no contribuyen a la predicción por ser muy específicas y por repetición("outlet_type")
    dataset = dataframe.drop(columns=['Item_Identifier', 'Outlet_Identifier',"Outlet_Type"])
    print(dataset)
    
    
    with open(modelo, 'rb') as archivo_pkl:
        modelo_cargado = pickle.load(archivo_pkl)
        
    
    prediction = modelo_cargado.predict(dataset)
    
    return prediction


In [131]:
df = "df.csv"
b = prediction(df,'model.pkl')
b

   Item_Weight  Item_Visibility Item_MRP  Outlet_Establishment_Year  \
0        20.75         0.007565        2                         21   

   Outlet_Size  Outlet_Location_Type  Outlet_Type_Grocery Store  \
0            1                     2                          0   

   Outlet_Type_Supermarket Type1  Outlet_Type_Supermarket Type2  \
0                              1                              0   

   Outlet_Type_Supermarket Type3  
0                              0  


0    1979.827531
Name: pred_Sales, dtype: float64